In [ ]:
import os, psutil

def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")

print_memory_usage()

In [ ]:
from geoarrow.pyarrow import io
import tempfile
import geopandas
import os


import shapely.geometry as geom
from d123.dataset.maps.gpkg.gpkg_map import GPKGMap
from d123.dataset.maps.map_datatypes import MapSurfaceType


temp_gpkg = "/home/daniel/d123_workspace/data/maps/carla_town01.gpkg"

print_memory_usage()
map_api = GPKGMap(temp_gpkg)
print_memory_usage()
map_api.initialize()
print_memory_usage()


table = map_api._gpd_dataframes[MapSurfaceType.LANE]

# import pyarrow as pa
# pa.table(table).schema

print_memory_usage()
inter = table.sindex.query(geom.box(-10, -10, 10, 10), predicate='intersects')


In [ ]:
import geoarrow.pyarrow as ga


from shapely import node
from shapely.strtree import STRtree
ga_array =  ga.as_geoarrow([str(i) for i  in pa_table["left_boundary"]])
# ga_array.wkb()
print_memory_usage()

STRtree([el.to_shapely() for el in ga_array], node_capacity=10)
print_memory_usage()

In [ ]:
import pyarrow.ipc as ipc
import pyarrow as pa

# Save to Arrow IPC file
with pa.OSFile("data.arrow", "wb") as sink:
    with ipc.new_file(sink, pa_table.schema) as writer:
        writer.write_table(pa_table)

# Load with memory mapping
with pa.memory_map("data.arrow", "r") as source:
    pa_table = ipc.open_file(source).read_all()

In [ ]:
import geoarrow.pandas as gap


gap.GeoArrowExtensionDtype

In [ ]:
print_memory_usage()
# Load with memory mapping
with pa.memory_map('data.arrow', 'r') as source:
    table_mmap = ipc.open_file(source).read_all()
print_memory_usage()

STRtree([el.to_shapely() for el in table_mmap["geometry"]], node_capacity=10)
print_memory_usage()





In [ ]:
from shapely.geometry import mapping
table_mmap["geom"].apply(lambda geom: mapping(geom.to_shapely()) if geom else None)